## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

###  1. Загрузка данных. 

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
data=pd.read_csv('data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Есть пропуски в столбцах days_employed и total_income

### 2. Предобработка данных

### Обработка пропусков

In [3]:
#посмотрим, что это за пропуски в 
#print(data[(data['total_income'].isnull())&(data['total_income'].isnull())].head(5))
#в обеих колонках пропуски выглядят как NaN. Заменяем их при помощи fillna() на медианные значения, 
#посчитанные по типу занятости
#использую именно медиану, так как разброс между значениями довольно большой

data.loc[data['income_type']=='пенсионер','days_employed']=data[data['income_type']=='пенсионер']['days_employed'].median()
data.loc[data['income_type']=='сотрудник','days_employed']=data[data['income_type']=='сотрудник']['days_employed'].median()
data.loc[data['income_type']=='предприниматель','days_employed']=data[data['income_type']=='предприниматель']['days_employed'].median()
data.loc[data['income_type']=='компаньон','days_employed']=data[data['income_type']=='компаньон']['days_employed'].median()
data.loc[data['income_type']=='студент','days_employed']=data[data['income_type']=='студент']['days_employed'].median()
data.loc[data['income_type']=='госслужащий','days_employed']=data[data['income_type']=='госслужащий']['days_employed'].median()

data.loc[data['income_type']=='пенсионер','total_income']=data[data['income_type']=='пенсионер']['total_income'].median()
data.loc[data['income_type']=='сотрудник','total_income']=data[data['income_type']=='сотрудник']['total_income'].median()
data.loc[data['income_type']=='предприниматель','total_income']=data[data['income_type']=='предприниматель']['total_income'].median()
data.loc[data['income_type']=='компаньон','total_income']=data[data['income_type']=='компаньон']['total_income'].median()
data.loc[data['income_type']=='студент','total_income']=data[data['income_type']=='студент']['total_income'].median()
data.loc[data['income_type']=='госслужащий','total_income']=data[data['income_type']=='госслужащий']['total_income'].median()




### Замена типа данных

In [4]:
#так как количество дней занятости в виде float выглядит странно, заменяем его на целочисленное. 
#Сумму дохода для анализа качества заемщика так же можно перевести в целочисленное, так как копейки в данном контексте несущественны
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-1574,42,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка жилья
1,1,-1574,36,среднее,1,женат / замужем,0,F,сотрудник,0,142594,приобретение автомобиля
2,0,-1574,33,Среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка жилья
3,3,-1574,32,среднее,1,женат / замужем,0,M,сотрудник,0,142594,дополнительное образование
4,0,365213,53,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


### Обработка дубликатов

In [5]:
#проанализируем содержимое столбцов, содержащих предположительно однотипные значения
print (data['education'].unique())
print (data['family_status'].unique())
print (data['purpose'].unique())
#в столбце "education" явно можно привести все к одному виду, прежде, чем удалять дубликаты
data['education']=data['education'].str.lower()
#print (data['education'].unique())
#удалить дубликаты
data=data.drop_duplicates().reset_index(drop= True)
#посчитать количество строк, которые остались после удаления дубликатов.
print(data.info())




['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного обра

### Лемматизация

In [6]:
strpurpose=data['purpose'].unique()
m = Mystem()

for purpose in strpurpose:
    lemmas = m.lemmatize(purpose)
    #lemmas = ' '.join(m.lemmatize(purpose))
    print (lemmas)

#print (data.head(10))


['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

### Категоризация данных

In [7]:
#проводим категоризацию возраста
def agetype (age):
    if age<=24:
        return 'моложе 24'
    elif ((age>24) & (age <=45)):
        return 'от 25 до 45'
    elif ((age>45) & (age <=65)): 
        return 'от 45 до 65'
    else:
        return 'старше 65'

data['age_type']=data['dob_years'].apply(agetype)


#проводим категоризацию доходов
def incomesumtype (incomesum):
    if incomesum==0:
         return 'нет данных о доходе или доход отсутствует'
    if incomesum<50000:
        return 'менее 50 тысяч'
    elif ((incomesum>=50000) & (incomesum <100000)):
        return 'от 50 до 100 тысяч'
    elif ((incomesum>=100000) & (incomesum <150000)):
        return 'от 100 до 150 тысяч'
    elif ((incomesum>=150000) & (incomesum <200000)): 
        return 'от 150 до 200 тысяч'
    else:
        return 'более 200 тысяч'

data['incomesum_type']=data['total_income'].apply(incomesumtype)
  
#проводим категоризацию по признаку наличия детей
def havechild (child):
    if child<=0:
        return 0
    else:
        return 1

data['child_type']=data['children'].apply(havechild)


data.head(5)
    





,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_type,incomesum_type,child_type
0,1,-1574,42,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка жилья,от 25 до 45,от 100 до 150 тысяч,1
1,1,-1574,36,среднее,1,женат / замужем,0,F,сотрудник,0,142594,приобретение автомобиля,от 25 до 45,от 100 до 150 тысяч,1
2,0,-1574,33,среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка жилья,от 25 до 45,от 100 до 150 тысяч,0
3,3,-1574,32,среднее,1,женат / замужем,0,M,сотрудник,0,142594,дополнительное образование,от 25 до 45,от 100 до 150 тысяч,1
4,0,365213,53,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,от 45 до 65,от 100 до 150 тысяч,0


In [8]:
#проводим категоризацию причин обращения за кредитом:
strpurpose=data['purpose'].unique()

m = Mystem()

def strlemma (purpose):
    lemmas = m.lemmatize(purpose)
   
    #if ('приобретение' in lemmas) & ('жилье' in lemmas):
    #    return 'покупка жилой недвижимости'
    if  ('автомобиль' in lemmas):
        return 'покупка автомобиль'
    if ('образование' in lemmas) :
        return 'образование'
    if ('свадьба' in lemmas) :
        return 'свадьба'
    if ('жилье' in lemmas) & ('ремонт' in lemmas) :
        return 'ремонт жилья'
    elif ('жилье' in lemmas):
        return 'покупка недвижимость'
    if ('недвижимость' in lemmas):
        return 'покупка недвижимость'
#    else:
#        return ' '.join(m.lemmatize(purpose))
    

data['purpose_type']=data['purpose'].apply(strlemma)
print (data['purpose_type'].unique())


['покупка недвижимость' 'покупка автомобиль' 'образование' 'свадьба'
 'ремонт жилья']


### 3. Выявление зависимостей

**? Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [9]:
#общее количество просрочек по выборке
data['debt'].agg(['count','mean'])

count    17883.000000
mean         0.095174
Name: debt, dtype: float64

In [10]:
#смотрим доли просрочек по группам заемщиков с детьми и без детей в целом
data.groupby('child_type')['debt'].agg(['count','mean'])



,count,mean
child_type,,
0,11303,0.092011
1,6580,0.100608


In [11]:
#смотрим доли просрочек по группам заемщиков без детей и с разным количеством детей
data.groupby('children')['debt'].agg(['count','mean'])

,count,mean
children,,
-1,47,0.021277
0,11256,0.092306
1,4290,0.100932
2,1841,0.103205
3,324,0.083333
4,40,0.100000
5,9,0.000000
20,76,0.105263


Промежуточный вывод: Вероятность просрочки выше у замещиков с детьми, но незначительно

**? Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [12]:
data.groupby('family_status')['debt'].agg(['count','mean']).sort_values(by='mean',ascending = False)


,count,mean
family_status,,
гражданский брак,3333,0.113411
Не женат / не замужем,2637,0.101251
женат / замужем,9916,0.091670
вдовец / вдова,845,0.074556
в разводе,1152,0.073785


Промежуточный вывод: Вероятность просрочки выше у заемщиков в гражданском браке

**? Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [13]:
data.groupby('incomesum_type')['debt'].agg(['count','mean']).sort_values(by='mean',ascending = False)


,count,mean
incomesum_type,,
от 50 до 100 тысяч,3,0.666667
от 100 до 150 тысяч,11850,0.104726
от 150 до 200 тысяч,6027,0.076157
более 200 тысяч,3,0.000000


Промежуточный вывод: Вероятность просрочки выше у заемщиков с доходом от 100 до 150 тысяч. Заемщиков с доходом от 50 до 100 тысяч слишком мало, наверное они все-таки не таки рискованные или нужна большая выборка, чтобы сделать по ним вывод

**? Как разные цели кредита влияют на его возврат в срок?**

In [14]:
data.groupby('purpose_type')['debt'].agg(['count','mean']).sort_values(by='mean',ascending = False)


,count,mean
purpose_type,,
свадьба,1563,0.113244
покупка автомобиль,3730,0.106434
образование,3546,0.103215
покупка недвижимость,8523,0.085416
ремонт жилья,521,0.065259


Промежуточный вывод: Вероятность просрочки выше у тех, кто берет кредит на свадьбу



### 4. Общий вывод


 В ходе работы анализировалась таблица с данными по заемщикам
Собранные данные неплохого качества, но вызвают вопросы ошибки в данных типа отрицательные значения в колонке "дети", отсутствующие уровни доходов и трудового стажа, которые мы заменили средними значениями
    
В целом по представленной выборке общая просрочка находится на неплохом уровне. Порядка 9,5%

**Цели кредитов**:
- образование
- свадьба
- покупка жилья
- ремонт жилья
- покупка автомобиля
  
Заемщики разных возрастов и уровня дохода, есть пенсионеры

    
**Наиболее рискованной группой замещиков явлются замщики не состоящие в официальном браке, берущие кредит на свадьбу* 



